# Install dependencies

In [4]:
! pip install transformers
! git clone https://github.com/aub-mind/arabert
! pip install -r arabert/requirements.txt
! pip install pytorch_pretrained_bert


     |████████████████████████████████| 2.6 MB 23.3 MB/s 
     |████████████████████████████████| 3.3 MB 53.2 MB/s 
     |████████████████████████████████| 636 kB 35.9 MB/s 
     |████████████████████████████████| 895 kB 46.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'arabert'...
remote: Enumerating objects: 530, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 530 (delta 167), reused 226 (delta 82), pack-reused 214
Receiving objects: 100% (530/530), 4.86 MiB | 21.64 MiB/s, done.
Resolving deltas: 100% (290/290), done.
     |████████████████████████████████| 126 kB 25.5 MB/s 
     |████████████████████████████████| 123 kB 29.8 MB/s 
     |████████████████████████████████| 131 kB 55.8 MB/s 
     |████████████████████████████████| 7.7 MB 44.4 MB/s 
     |████████████████████████████████| 79 kB 5

# Imports

In [5]:
from transformers import AutoTokenizer, AutoModel,AutoModelForMaskedLM
from arabert.preprocess import ArabertPreprocessor
import torch
from pytorch_pretrained_bert import BertTokenizer,BertForMaskedLM
import pandas as pd
import math

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')



In [11]:
from tabulate import tabulate


In [ ]:
import sys
sys.path.insert(0, './src')
print(sys.path)
from tabulate import tabulate
import argparse
from .src.BERT2LM import utils


# Data Configurations

In [25]:
QData = [{
    "Sentence":"What’s the Eid al-Adha [TGT] DATE [TGT]",
        "Query":"DATE",
        "Translation":[
                      {
          "Rank": -1,
          "Meaning":"establish or ascertain the date of (an object or event).",
          "English":"What’s the Eid al-Adha date",
          "Arabic":"ما هو تاريخ عيد الأضحى",          
        },
        {
          "Rank": 1,
          "Meaning":"the specified day of the month",
          "English":"What’s the Eid al-Adha the specified day of the month",
          "Arabic":"ما هو عيد الأضحى في اليوم المحدد من الشهر",          
        },
        {
          "Rank": 2,
          "Meaning":"the particular day, month, or year (usually according to the Gregorian calendar) that an event occurred",
          "English":"What’s the Eid al-Adha the particular day, month, or year (usually according to the Gregorian calendar) that an event occurred",
          "Arabic":"ما هو عيد الأضحى في اليوم أو الشهر أو السنة المحددة (عادةً وفقًا للتقويم الميلادي) الذي وقع فيه الحدث",          
        },
        {
          "Rank": 3,
          "Meaning":"a particular day specified as the time something happens",
          "English":"What’s the Eid al-Adha a particular day specified as the time something happens",
          "Arabic":"ما هو عيد الأضحى في يوم معين محدد على أنه وقت حدوث شيء ما",          
        },
        {
          "Rank": 4,
          "Meaning":"assign a date to; determine the (probable) date of",
          "English":"What’s the Eid al-Adha assign a date to; determine the (probable) date of",
          "Arabic":"ما هو عيد الأضحى لقاء رتبته مسبقا",          
        }
        ]
    },
    {
       "Sentence":"I am [TGT] WATCH [TGT] ING you eating Hummus",
        "Query":"WATCH",
             "Translation":[
                      {
          "Rank": -1,
          "Meaning":"look at or observe attentively over a period of time",
          "English":"I am watching you eating Hummus",
          "Arabic":"أنا أشاهدك تأكل الحمص",          
        },
        {
          "Rank": 1,
          "Meaning":"look attentively",
          "English":"I am look attentively  you eating Hummus",
          "Arabic":"أنا أنظر باهتمام وأنت تأكل الحمص",          
        },
        {
          "Rank": 2,
          "Meaning":"follow with the eyes or the mind",
          "English":"I am follow with the eyes or the mind  you eating Hummus",
          "Arabic":"أنا أتابع بالعيون أو العقل تأكل الحمص",          
        },
        {
          "Rank": 3,
          "Meaning":"observe with attention",
          "English":"I am observe with attention you eating Hummus",
          "Arabic":"ألاحظ باهتمام أنك تأكل الحمص",          
        },
        {
          "Rank": 4,
          "Meaning":"purposeful surveillance to guard or observe",
          "English":"I am a purposeful surveillance to guard or observe you eating Hummus",
          "Arabic":"أنا مراقب هادف لأحرسك أو أراقبك وأنت تأكل الحمص",          
        }
        ]
    },
        {
       "Sentence":"i am [TGT] FAST [TGT] ing in ramadan",
        "Query":"FAST",
             "Translation":[
                      {
          "Rank": -1,
          "Meaning":"abstain from all or some kinds of food or drink, especially as a religious observance.",
          "English":"i am fasting in ramadan",
          "Arabic":"انا صائم رمضان",          
        },
        {
          "Rank": 1,
          "Meaning":"abstain from certain foods, as for religious or medical reasons",
          "English":"i am abstain from certain foods, as for religious or medical reasons in ramadan",
          "Arabic":"أنا أمتنع عن بعض الأطعمة ، لأسباب دينية أو طبية في رمضان",          
        },
        {
          "Rank": 2,
          "Meaning":"abstain from eating",
          "English":"i am abstain from eating in ramadan",
          "Arabic":"أنا أمتنع عن الأكل في رمضان",          
        },
        {
          "Rank": 3,
          "Meaning":"abstaining from food",
          "English":"i am abstaining from food in ramadan",
          "Arabic":"أنا أمتنع عن الأكل في رمضان",          
        },
        {
          "Rank": 4,
          "Meaning":"unrestrained by convention or morality",
          "English":"i am unrestrained by convention or morality in ramadan",
          "Arabic":"أنا غير مقيد بالأعراف والأخلاق في رمضان",          
        }
        ]
    }
    
    ]



# Initialize Model and Tokenizer 

In [6]:
# model_name = "aubmindlab/bert-base-arabertv2"
model_name="aubmindlab/bert-large-arabertv02"
lm_bertMaskedLM = AutoModelForMaskedLM.from_pretrained(model_name)
lm_tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at aubmindlab/bert-large-arabertv02 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Implementations 

In [7]:
def get_score(sentence):
    tokenize_input = lm_tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([lm_tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=lm_bertMaskedLM(tensor_input)
    #print(predictions[0])
    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(predictions[0].squeeze(),tensor_input.squeeze()).data 
    #print(loss.data)
    return math.exp(loss)

# RUN

In [17]:
# s0 = get_score("ما هو تاريخ عيد الأضحى")
# s1 = get_score("ما هو عيد الأضحى في اليوم المحدد من الشهر")
# s2 = get_score("ما هو عيد الأضحى في اليوم أو الشهر أو السنة المحددة (عادةً وفقًا للتقويم الميلادي) الذي وقع فيه الحدث")
# s3 = get_score("ما هو عيد الأضحى في يوم معين محدد على أنه وقت حدوث شيء ما")
# s4 = get_score("ما هو عيد الأضحى لقاء رتبته مسبقا")

s0 = get_score("انا صائم رمضان")
s1 = get_score("أنا أمتنع عن بعض الأطعمة ، لأسباب دينية أو طبية في رمضان")
s2 = get_score("أنا أمتنع عن الأكل في رمضان")
s3 = get_score("أنا أمتنع عن الأكل في رمضان")
s4 = get_score("أنا غير مقيد بالأعراف والأخلاق في رمضان")
s = s0 +s1+s2+s3+s4
print(s0,s0/s)
print(s1,s1/s)
print(s2,s2/s)
print(s3,s3/s)
print(s4,s4/s)


35928.818845484486 0.9995111221576987
2.78182727660848 7.73882190463738e-05
4.915089444883785 0.00013673387337582004
4.915089444883785 0.00013673387337582004
4.961388510512667 0.00013802187650335376


In [9]:
for q in QData:
  rankList = []
  sum = 0
  if q["Sentence"] == "":
    continue
  else:
    for t in q["Translation"]:
      s = get_score(t["Arabic"])
      rankList.append({"meaning":t["Meaning"],"score":s})
      sum+=s
    for r in rankList:
      print("{} : {}".format(r["meaning"],r["score"]/sum))
      #print("{} : {}".format(t["Meaning"],s))
      # q["Score"] = s
      # rankDic.append(q)
  #print(rankDic)


the event date : 0.9972292488969371
the specified day of the month : 0.0010806762809566993
the particular day, month, or year (usually according to the Gregorian calendar) that an event occurred : 0.0003500312610006416
a particular day specified as the time something happens : 0.00048813059223069925
assign a date to; determine the (probable) date of : 0.0008519129688748321


In [26]:
for q in QData:
  rankList = []
  sum = 0
  if q["Sentence"] == "":
    continue
  else:
    print(q["Sentence"])
    for t in q["Translation"]:
      s = get_score(t["Arabic"])
      t["Score"] =s

      rankList.append(t)
      sum+=s
    # for r in rankList:
      # print("{} : {}".format(r["Meaning"],r["Score"]/sum))
    newlist = sorted(rankList, key=lambda k: k['Score'],reverse=True) 

    tab = []
    for i,t in enumerate(newlist):
      av = t["Score"]/sum
      tab.append([f"{i}.", t["Rank"], t["English"],t["Score"], f"{av:.5f}"])
      tab.append([f"{i}.", t["Rank"], t["Arabic"],t["Score"], f"{av:.5f}"])
      tab.append([f"{i}.", t["Rank"], t["Meaning"],t["Score"], f"{av:.5f}"])


    print(tabulate(tab,headers=["No.", "WSD Rank", "Definition", "Score","Avarege"]))


    # print(tabulate(
    #     [[f"{i+1}.", t["Rank"], t["Meaning"],t["Score"], f"{t["Score"]/sum:.5f}"] for i, t in enumerate(rankList)],
    #     headers=["No.", "WSD Rank", "Definition", "Score","Avarege"])


What’s the Eid al-Adha [TGT] DATE [TGT]
  No.    WSD Rank  Definition                                                                                                                           Score    Avarege
-----  ----------  ------------------------------------------------------------------------------------------------------------------------------  ----------  ---------
    0          -1  What’s the Eid al-Adha date                                                                                                     3862.2        0.99723
    0          -1  ما هو تاريخ عيد الأضحى                                                                                                          3862.2        0.99723
    0          -1  establish or ascertain the date of (an object or event).                                                                        3862.2        0.99723
    1           1  What’s the Eid al-Adha the specified day of the month                                           